### 0.Introduction
This notebook will guide you to run spatial-queue based traffic simulations to simulate the traffic flow during a hypothesized evacuation scenario in North Berkeley. Specifically, you will run a base case scenario and determine where the traffic bottlenecks are in the base scenario. To identify the bottleneck locations, you will need to visualize your output in QGIS. Next, you will need to specify your desired location of implementing contraflow to ease the bottleneck.

There is no need to change the code other than places with ###YOUR_CODE[NAME]_HERE###.

### 1.Download input files and required modules

In [ ]:
# retrieve the sp code
!rm -rf sp && mkdir sp
!wget "https://github.com/UCB-CE170a/Fall2020/raw/master/traffic_data/liblsp.so" -O sp/liblsp.so
!wget "https://raw.githubusercontent.com/UCB-CE170a/Fall2020/master/traffic_data/interface.py" -O sp/interface.py

# retrieve the spatial queue model
!wget "https://raw.githubusercontent.com/UCB-CE170a/Fall2020/master/traffic_data/queue_model.py" -O queue_model.py

# retrieve the road network
!rm -rf traffic_inputs && mkdir traffic_inputs
!wget "https://raw.githubusercontent.com/UCB-CE170a/Fall2020/master/traffic_data/berkeley_links.csv" -O traffic_inputs/berkeley_links.csv
!wget "https://raw.githubusercontent.com/UCB-CE170a/Fall2020/master/traffic_data/berkeley_nodes.csv" -O traffic_inputs/berkeley_nodes.csv
!wget "https://raw.githubusercontent.com/UCB-CE170a/Fall2020/master/traffic_data/od_20pn.csv" -O traffic_inputs/od_20pn.csv
!wget "https://raw.githubusercontent.com/UCB-CE170a/Fall2020/master/traffic_data/od_10pn.csv" -O traffic_inputs/od_10pn.csv
!wget "https://raw.githubusercontent.com/UCB-CE170a/Fall2020/master/traffic_data/od_50pn.csv" -O traffic_inputs/od_50pn.csv

# create output directory
!rm -rf traffic_outputs
!mkdir -p traffic_outputs
!mkdir -p traffic_outputs/t_stats
!mkdir -p traffic_outputs/node_stats
!mkdir -p traffic_outputs/link_stats

### 2.Import modules and libraries

In [ ]:
import time 
import random
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
%matplotlib inline
random.seed(0)
np.random.seed(0)

### spatial queue model
from queue_model import Node, Link, Agent, Simulation

### Task 1. Run the base scenario
In this section, we will run the base scenario where no contraflow is implemented.

Step1: Read data and set some simulation parameters. 

In [ ]:
scenario = 'base'
yourname = 'tester' ### YOUR_NAME_HERE avoid space ###
scenario_name = '{}-{}'.format(scenario, yourname)

# simulation duration in seconds, specify at which time step to stop the simulation. It needs to be long enough for all evacuees to arrive at the safe destination.
t_end = 3601

# vehicle rerouting frequency (second per reroute). Default value is 3600s or 1 hour -- baseically no rerouting
# do not change this value for results submitted on September 30. But you can change it for your report.
reroute_frequency = 1800

# read data
nodes_df = pd.read_csv('traffic_inputs/berkeley_nodes.csv')
links_df = pd.read_csv('traffic_inputs/berkeley_links.csv')
od_df = pd.read_csv('traffic_inputs/od_20pn.csv') ### you can use od_10pn.csv (lower demand) or od_50pn.csv (higher demand) for your report (but not for results submitted on September 30).

Step2: initialize the simulation

In [ ]:
# remove vehicles from the simulation if no path can be found for it
def remove_no_path_agents(simulation):
  cannot_find_path = []
  for vehicle_id, vehicle in simulation.all_agents.items():
      routing_status = vehicle.get_path( g=simulation.g )
      if routing_status == 'no_path_found':
          cannot_find_path.append(vehicle_id)

  for vehicle_id in cannot_find_path:
      del simulation.all_agents[vehicle_id]
      
  print('# o-d pairs whose paths cannot be found: {}'.format(len(cannot_find_path)))
  print('# o-d pairs/trips {}'.format(len(simulation.all_agents)))
  return simulation

# initialize the spatial-queue model
def init_sq_simulation(nodes_df, links_df, od_df):

  simulation = Simulation()
  simulation.create_network(nodes_df, links_df)
  simulation.create_demand(od_df)

  simulation = remove_no_path_agents(simulation)
  return simulation

In [ ]:
simulation = init_sq_simulation(nodes_df,links_df,od_df)

Step3: Run the spatial queue simulation

In [ ]:
# run the spatial-queue simulation for one time step
def single_step_sq_sim(simulation,t,reroute_frequency):
  ### load agents
  for agent_id, agent in simulation.all_agents.items(): 
    agent.load_trips(t)
    ### reroute
    if (t>0) and (t%reroute_frequency == 0):
        routing_status = agent.get_path( g=simulation.g )
  ### run link model
  for link_id, link in simulation.all_links.items():
    link.run_link_model(t)
  ### run node model
  node_ids_to_run = set([link.end_nid for link in simulation.all_links.values() if len(link.queue_veh)>0])
  for node_id in node_ids_to_run:
    node = simulation.all_nodes[node_id] 
    node.run_node_model(t)
  return simulation

# count the number of evacuees that have successfully reach their destination
def arrival_counts(t,simulation,save_path):
  arrival_cnts = np.sum([1 for a in simulation.all_agents.values() if a.status=='arr'])
  print('At {} seconds, {} evacuees successfully reached the destination'.format(t, arrival_cnts))
  if arrival_cnts == len(simulation.all_agents):
      print("all agents arrive at destinations at time {} seconds.".format(t))
      return False
  with open(save_path, 'a') as t_stats_outfile:
      t_stats_outfile.write("{},{}".format(t, arrival_cnts) + "\n")
  return True

# write a csv file that contains the numbers of queuing and running vehicles on each link
def write_link_outputs(simulation,save_path):
  link_output = pd.DataFrame([(link.lid, len(link.queue_veh), len(link.run_veh), link.geometry) for link in simulation.all_links.values() if link.ltype[0:2]!='vl'], columns=['link_id', 'queue_vehicle_count', 'run_vehicle_count', 'geometry'])
  link_output = link_output[(link_output['queue_vehicle_count']>0) | (link_output['run_vehicle_count']>0)].reset_index(drop=True)
  link_output.to_csv(save_path, index=False)

# write a csv file that contains the numbers of vehicles that have not departed and waiting at each node
def write_node_outputs(simulation,save_path):
  node_predepart = pd.DataFrame([(agent.cle, 1) for agent in simulation.all_agents.values() if (agent.status in [None, 'loaded'])], columns=['node_id', 'predepart_cnt']).groupby('node_id').agg({'predepart_cnt': np.sum}).reset_index()
  if node_predepart.shape[0]>0:
      node_predepart = node_predepart.merge(nodes_df[['node_id', 'lat', 'lon']], how='left', on='node_id')
      node_predepart.to_csv(save_path, index=False)



In [ ]:
def spatial_queue_simulation(t_end,simulation,scenario_name,reroute_frequency):
  # paths 
  arrival_output_path = 'traffic_outputs/t_stats/arrivals_{}.csv'.format(scenario_name)
  with open(arrival_output_path, 'w') as t_stats_outfile:
      t_stats_outfile.write("t,arrival_count"+"\n")

  # iterate through each time step
  for t in range(t_end):
      # run the spatial-queue simulation for one step
      simulation = single_step_sq_sim(simulation,t,reroute_frequency)

      # output time-step results every 100 seconds
      if t%100 == 0:
        if not arrival_counts(t,simulation,arrival_output_path):
          break
        link_output_path = 'traffic_outputs/link_stats/l{}_at_{}.csv'.format(scenario_name, t)
        node_output_path = 'traffic_outputs/node_stats/n{}_at_{}.csv'.format(scenario_name, t)
        write_link_outputs(simulation,link_output_path)
        write_node_outputs(simulation,node_output_path)

  print ("simulation completed")
  return simulation

In [ ]:
spatial_queue_simulation(t_end, simulation, scenario_name, reroute_frequency)

### Task 1.b Visualize the results
There are at least two types of visualizations that you can make based on the output files from running the simulation code above:

1. Arrival curve. This is a line plot showing the number of arrived vehicles at different time steps. The code block below shows an example of how to make the arrival plot.

2. Queueing status and departure status map. Suppose we want to visualize the queueing status around 20 minutes after the onset of the evacuation under the base scenario in QGIS, we just need to load the file `link_stats_base_t1200.csv` to QGIS and choose a suitable categorized legend. Similarly, we can also load `node_stats_base_t1200.csv` to the same map to display the number of vehicles that are still at their origin node (cannot departure because the road in front of their origin is too congested). Based on our experience, most vehicles should be able to depart pretty early, so `node_stats_base_t[...].csv` is likely to be empty.

These visualizations can help you to get an intuitive idea of whether your strategy is effective.

In [ ]:
### plot the arrival curve
fig, ax = plt.subplots(figsize=(10,5))

### base scenario arrival
t_stats_base_df = pd.read_csv('traffic_outputs/t_stats/arrivals_{}-{}.csv'.format('base', yourname))
ax.plot(t_stats_base_df['t'], t_stats_base_df['arrival_count'], label='base')

ax.set_xlabel('Time since the start of the evacuation')
ax.set_ylabel('# arrived vehicles')
ax.set_xlim([0, 3600])
plt.legend()
plt.show()

In [ ]:
### run this command to put all the outputs in a compressed file
### you can then download the file to your local computer for visualization
!tar czvf traffic_outputs.tar.gz traffic_outputs

### Task 2. Contraflow scenario

Step1: read files. You need to upload your contraflow links file CSV to `traffic_inputs/contraflow_links_[your_name].csv`. Refer to [the Github page](https://github.com/UCB-CE170a/Fall2020/edit/master/homeworks/hw2/README.md) for formats and requirements of this contraflow file.

In [ ]:
nodes_df = pd.read_csv('traffic_inputs/berkeley_nodes.csv')
links_df = pd.read_csv('traffic_inputs/berkeley_links.csv')
od_df = pd.read_csv('traffic_inputs/od_20pn.csv')

scenario = 'contraflow'
scenario_name = '{}-{}'.format(scenario, yourname)

contraflow_path = 'traffic_inputs/contraflow_links_{}.csv'.format(yourname)
contraflow_links_df = pd.read_csv(contraflow_path)
print('Your choice of contraflow locations are \n')
display(contraflow_links_df.head())

links_df = links_df.merge(contraflow_links_df[['link_id', 'new_lanes']], how='left', on='link_id')
links_df['lanes'] = np.where(np.isnan(links_df['new_lanes']), links_df['lanes'], links_df['new_lanes'])


step2: initialize simulation

In [ ]:
simulation = init_sq_simulation(nodes_df, links_df, od_df)

step3: run simulation 

In [ ]:
spatial_queue_simulation(t_end, simulation, scenario_name, reroute_frequency)

In [ ]:
### plot the arrival curve
fig, ax = plt.subplots(figsize=(10,5))

### base scenario arrival
t_stats_base_df = pd.read_csv('traffic_outputs/t_stats/arrivals_{}-{}.csv'.format('base', yourname))
ax.plot(t_stats_base_df['t'], t_stats_base_df['arrival_count'], label='base')

## contraflow scenario arrival
t_stats_contraflow_df = pd.read_csv('traffic_outputs/t_stats/arrivals_{}-{}.csv'.format('contraflow', yourname))
ax.plot(t_stats_contraflow_df['t'], t_stats_contraflow_df['arrival_count'], label='contraflow')

ax.set_xlabel('Time since the start of the evacuation')
ax.set_ylabel('# arrived vehicles')
ax.set_xlim([0, 3600])
plt.legend()
plt.show()